### Run the copy of our database functions

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


| biginteger type of id      | autoincrement type of id |
| ----------- | ----------- |
| <pre>m = DiscordUser<br>e = DiscordEmote<br>g = DiscordGuilds<br>l = LeagueAccount<br>d = DotaAccount<br>a = AFKNote<br>s = mygamerdata</pre>      | <pre>r = RemindersNote<br>t = ToDoNote<br>w = WarningData<br>u = MuteData<br>c = ChannelsAssignmentData<br>sr = serverrules<br>rr = realrule<br>tg = tagsdb</pre>          |

In [1]:
# %run ./utils/database.py
from utils import database as db
import pandas as pd
def get_df(dbclass):
    return pd.read_sql_table(dbclass.__tablename__, db.engine, index_col='id')

### Now our draft things

In [31]:
df = get_df(db.w)

In [6]:
print(sum(get_value(e, 869827100926951465, 'month_array')))

1


In [ ]:
"""
for index, row in df.iterrows():
    value = get_value(e, index, 'month_array')
    df.loc[index, 'month_array'] = value
"""

In [7]:
remove_row(e, 869827100926951465)

In [5]:
with db.session_scope() as ses:
    for row in ses.query(db.sr):
        row.id = 0

In [2]:
db.tg.__table__.drop(db.engine)

In [5]:
def add_column(enginex, table_name, columnx):
    column_name = columnx.compile(dialect=enginex.dialect)
    column_type = columnx.type.compile(enginex.dialect)
    #column_default = column.default.arg
    enginex.execute(f'ALTER TABLE "{table_name}" ADD COLUMN {column_name} {column_type}') #DEFAULT #{column_default}')

column = db.Column('can_make_tags', db.Boolean, default=True)
add_column(db.engine, 'GuildMembers', column)

In [11]:
def drop_column(enginex, table_name, column_name):
    enginex.execute(f'ALTER TABLE "{table_name}" DROP COLUMN {column_name}')

drop_column(engine, 'GuildMembers', 'name')

In [13]:
# options to see the whole table 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [10]:
# option to reset these things
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

In [24]:
from difflib import SequenceMatcher

textes = [
  'hallo hello allo ello ell', 
  'Hallo Hello hello hello hello hello hello Agalo Hellao',
  'HALLO HALLO HALLO HALLO HALLO',
  'Hello HeLlA ALHO EHLA ELHA'
]

query_string = 'hello'

def similar(a, b):
    return SequenceMatcher(a=a.lower(), b=b.lower()).ratio()

for i in textes:
    s = similar(query_string, i)
    print(s)
    # print(sum(n for i,j,n in s.get_matching_blocks()) / float(len(query_string)))

0.3333333333333333
0.1694915254237288
0.23529411764705882
0.3225806451612903


In [19]:
import re
from typing import Optional
from typing_extensions import Self
import datetime
from dateutil.relativedelta import relativedelta

class ShortTime:
    compiled = re.compile(
        """
           (?:(?P<years>[0-9])(?:years?|y))?             # e.g. 2y
           (?:(?P<months>[0-9]{1,2})(?:months?|mo))?     # e.g. 2months
           (?:(?P<weeks>[0-9]{1,4})(?:weeks?|w))?        # e.g. 10w
           (?:(?P<days>[0-9]{1,5})(?:days?|d))?          # e.g. 14d
           (?:(?P<hours>[0-9]{1,5})(?:hours?|h))?        # e.g. 12h
           (?:(?P<minutes>[0-9]{1,5})(?:minutes?|m))?    # e.g. 10m
           (?:(?P<seconds>[0-9]{1,5})(?:seconds?|s))?    # e.g. 15s
        """,
        re.VERBOSE,
    )

    def __init__(self, argument: str, *, now: Optional[datetime.datetime] = None):
        match = self.compiled.fullmatch(argument)
        if match is None or not match.group(0):
            raise ValueError('invalid time provided')

        data = {k: int(v) for k, v in match.groupdict(default=0).items()}
        now = now or datetime.datetime.now(datetime.timezone.utc)
        self.dt: datetime.datetime = now + relativedelta(**data)

    @classmethod
    async def convert(cls, argument: str) -> Self:
        return cls(argument)
    
xd = ShortTime('1y')
xd.dt

ValueError: invalid time provided

In [9]:
from pytimeparse import parse
from datetime import timedelta

string = ' seconds'
while(True):
    string = '1' + string
    print(len(string), end='\r')
    seconds = parse(string)
    print(seconds)
    delta = timedelta(seconds=seconds)

1
11
111
1111
11111
111111
1111111
11111111
111111111
1111111111
11111111111
111111111111
1111111111111
11111111111111
111111111111111


OverflowError: days=1286008230; must have magnitude <= 999999999